Лабораторна робота 8. Варіант 3.
Тема: Транспортна задача.

Додаємо початкові пакети для викоання завдання.

In [17]:
import numpy as np
import pandas as pd
from pulp import LpMinimize, LpProblem, LpVariable, lpSum
from scipy.optimize import linear_sum_assignment


* <h3>Завдання 1(задача 7). Метод північно-західного кута.</h3>
Створюємо метод північно-західного кута, який приймає на вхід елементи матриці(значення маршрутів) і двої масивів(потреби і постачання).

In [29]:
# Метод північно-західного кута
def northwest_corner_method(supply, demand, costs):
    rows, cols = len(supply), len(demand)
    allocation = np.zeros((rows, cols))

    i, j = 0, 0

    while i < rows and j < cols:
        quantity = min(supply[i], demand[j])
        allocation[i, j] = quantity
        supply[i] -= quantity
        demand[j] -= quantity

        if supply[i] == 0:
            i += 1
        if demand[j] == 0:
            j += 1

    return allocation

# Метод обчислення загальної вартості
def calculate_total_cost(allocation, costs):
    total_cost = np.sum(allocation * costs)
    return total_cost


Введення початкових данних. Створення матриці та двух масивів - потреби(b) і постачання(a). Виведення оптимального плану.

In [31]:

# Початкові дані
supply = [230, 250, 170, 150]
demand = [140, 90, 160, 110, 150]

costs = np.array([
    [40, 19, 25, 25, 35],
    [49, 26, 27, 18, 38],
    [46, 27, 36, 40, 45],
    [140, 90, 160, 110, 150]
])

# Виклик методу
allocation = northwest_corner_method(supply, demand, costs)

# Виведення кінцевого результату(оптимальний план)
print("Оптимальний план(таблиця маршрутів):")
'''print(allocation)'''
df = pd.DataFrame(allocation.reshape(4, 5))
# змінюємо індексацію
df.index = np.arange(1, df.shape[0] + 1)
df.columns = np.arange(1, df.shape[1] + 1)
display(df)

print("Загальна вартість(F):", calculate_total_cost(allocation, costs))

Оптимальний план(таблиця маршрутів):


,1,2,3,4,5
1,140.0,90.0,0.0,0.0,0.0
2,0.0,0.0,160.0,90.0,0.0
3,0.0,0.0,0.0,20.0,150.0
4,0.0,0.0,0.0,0.0,0.0


Загальна вартість(F): 20800.0


* <h3> Завдання 2(задача 4). Метод потенціалів. </h3>
Введення початкових значень для матриці маршрутів(вартості) та масивів постачання та потреб. Визначаємо довжини масивів.

In [34]:
# Початкові дані
supply = [300, 250, 200] # Постачання
demand = [210, 150, 120, 135, 135] # Потреби

costs = [
    [4, 8, 13, 2, 7],
    [9, 4, 11, 9, 17],
    [3, 16, 10, 1, 4]
]

suppliers = range(len(supply))
consumers = range(len(demand))


Метод для знаходження потенціалів та визначення від'ємних та додатніх значень.

In [35]:

costs = makeDict ([suppliers, consumers], costs, 0)

prob = LpProblem("Transportation_Problem", LpMinimize)

routes = [(i, j) for i in suppliers for j in consumers]

vars = LpVariable.dicts("Route", (suppliers, consumers), 0, None, LpInteger)

prob += (
    lpSum([vars[w][b] * costs[w][b] for (w, b) in routes]),
    "Sum_of_Transporting_Costs",
)

for s in suppliers:
    prob += (
        lpSum([vars[s][b] for b in consumers]) <=supply[s],
        "sum_of_Products_out_of_suppliers_%s" % s,
    )

for c in consumers:
    prob += (
        lpSum([vars[w][c] for w in suppliers]) >=demand[c],
        "Sum_of_Products_into_consumer%s" % c,
    )


Виклик методів та виведення результатів.

In [37]:

prob.solve()

results = [v.varValue for v in prob.variables()]

print("Оптимальний план(таблиця маршрутів): ")
df = pd.DataFrame(np.array(results).reshape(3, 5))
df.index = np.arange(1, df.shape[0] + 1)
df.columns = np.arange(1, df.shape[1] + 1)
display(df)

print("Загальна вартість(F):", value(prob.objective))


Оптимальний план(таблиця маршрутів): 


,1,2,3,4,5
1,165.0,0.0,0.0,135.0,0.0
2,0.0,150.0,100.0,0.0,0.0
3,45.0,0.0,20.0,0.0,135.0


Загальна вартість(F): 3505.0
